Title of this file may need to change.
This file will calculate the 3rd and 4th columns that are needed for ARCiS to function

In [3]:
import numpy as np
from astropy import units as u
import csv
import configparser
import os
import subprocess
from spectres import spectres

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt

# CONFIG
LIFE_spectral_resolution = 100

In [41]:
def ARCiS_file_generator(LIFEsim_filename,LIFE_spectral_resolution):
    """Take one LIFEsim spectrum, add colums for noise and bandwith, and save it in ARCiS folder"""
    
    # get clean LIFEsim data [Jy]
    data_path = "../output/LIFEsim/cleanspectra/" + LIFEsim_filename
    bandcenter = np.genfromtxt(data_path,skip_header=0)[:,0] # microns
    bandflux = np.genfromtxt(data_path,skip_header=0)[:,1] # Jansky

    # get observed LIFEsim noise and bandwidths
    data_path = "../output/LIFEsim/observed_spectra/" + LIFEsim_filename
    bins = np.genfromtxt(data_path,skip_header=2,delimiter=",")[:,0] # microns
    SN = np.genfromtxt(data_path,skip_header=2,delimiter=",")[:,1] # Jansky
    
    # calculate spectral resolution per bin; should be very close to LIFEsim R
    spectral_resolution = []
    for i in range(len(bins)):
        if i == 0:
            spectral_resolution.append(bins[i]/(bins[i+1]-bins[i]))
        else:
            spectral_resolution.append(bins[i]/(bins[i]-bins[i-1]))

    # change original AGNI spectrum to LIFEsim bins
    new_bandflux = spectres(bins,bandcenter,bandflux)

    # draw random noise from LIFEsim SN
    noise = np.random.normal(0,new_bandflux/SN,size=new_bandflux.shape)

    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bins,new_bandflux,color='black', label='Input Spectrum')
    frame1.set_xlabel('wavelength (microns)')
    frame1.set_ylabel('Janskys')
    frame1.set_yscale('log')
    frame1.fill_between(bins,
                 new_bandflux-new_bandflux/SN,
                 new_bandflux+new_bandflux/SN,
                 color='black', alpha=0.1, label='1-$\sigma$')
    skip_values = 6
    frame1.scatter(bins[0::skip_values], new_bandflux[0::skip_values]+noise[0::skip_values],
            color='black', marker='.', label='Sim. Observation')
    frame1.errorbar(bins[0::skip_values], new_bandflux[0::skip_values]+noise[0::skip_values],
             yerr=abs(noise[0::skip_values]), color='black', capsize=4, ls='none')
    frame1.legend()

    # save images
    plt.ioff() # supress notebook output
    filename = "../output/ARCiS/input_spectra/figures/" + "ARCiS_input_" + LIFEsim_filename[15:-4] + '.png'
    fig.savefig(filename)
    
    
    # write to text files
    file_destination = "../output/ARCiS/input_spectra/textfiles/" + "ARCiS_input_" + LIFEsim_filename[15:-4] + ".txt"
    with open(file_destination, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(bins,new_bandflux,noise,spectral_resolution))
    

In [42]:
data_path = "../output/LIFEsim/cleanspectra"
files = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
for i in range(len(files)):
    ARCiS_file_generator(files[i],LIFE_spectral_resolution)